In [2]:
import torch
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = datasets.MNIST('mnist_data/', download=True, train=True, transform=transform, target_transform=None)
testset = datasets.MNIST('mnist_data/', download=True, train=False, transform=transform, target_transform=None)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle=False)


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

class Model(nn.Module):
    def __init__(self,imgWidth,imgHeight,numClasses):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(imgWidth*imgWidth, 10)  
        self.fc2 = nn.Linear(10,numClasses)
        self.softmax = nn.Softmax()

    def forward(self, img):
        img = img.view(img.size(0), -1)  
        outputOfLayer1 = self.fc1(img)
        finalOutput = self.softmax(self.fc2(outputOfLayer1))
        return finalOutput
    
def calculate_accuracy(model, dataloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total


if __name__ == "__main__":
    imgWidth = 28
    imgHeight = 28
    numClasses = 10

    model = Model(imgWidth, imgHeight, numClasses).to(device='cpu')

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.RMSprop(model.parameters(), lr = 0.01)

    numEpochs = 10
    for epoch in range(numEpochs):
        running_loss = 0.0
        for dataSample in trainloader:
            image, label = dataSample
            optimizer.zero_grad()  
            output = model(image)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {running_loss / len(trainloader)}")

    print('Training Finished')

    test_accuracy = calculate_accuracy(model, testloader)

    print(f"Test Accuracy: {100 * test_accuracy:.2f}%")



Epoch 1, Loss: 1.7508096286991257
Epoch 2, Loss: 1.6594927723982187
Epoch 3, Loss: 1.6658229249626844
Epoch 4, Loss: 1.6589051201653633
Epoch 5, Loss: 1.663610090070696
Epoch 6, Loss: 1.6616260241597955
Epoch 7, Loss: 1.6650193611950255
Epoch 8, Loss: 1.6589194907308387
Epoch 9, Loss: 1.6611088570247072
Epoch 10, Loss: 1.6579960294878051
Epoch 11, Loss: 1.6637815784797994
Epoch 12, Loss: 1.657273750315343
Epoch 13, Loss: 1.661971983624928
Epoch 14, Loss: 1.6578729831333607
Epoch 15, Loss: 1.6587596512806695
Epoch 16, Loss: 1.6616928255888446
Epoch 17, Loss: 1.6652932549590496
Epoch 18, Loss: 1.6520054069663417
Epoch 19, Loss: 1.6561961107925056
Epoch 20, Loss: 1.6584258953899718
Training Finished
Test Accuracy: 80.68%
